In [5]:
import numpy as np

In [13]:
# interest_region = {'Pg_f': [1000000.0, 2000000.0],
#                    'Tg_Tmax': [2000, 4500]}

interest_region = {'Pg_f': [0.2, 0.3],
                   'Tg_Tmax': [0.4, 0.7]}

coefficients = {
                'value': 1
                }
a, c = [30, 10]

In [14]:
def interest(y_hat: np.ndarray) -> float:
    res = 1
    for t, val in enumerate(interest_region.values()):
        ll, ul = val
        if y_hat.shape == (1, 2):
            v = -0.5 + (y_hat[0][t] - ll) / (ul - ll)
        else:
            v = -0.5 + (y_hat[t] - ll) / (ul - ll)
        partial_res = coefficients['value'] + (1 / (1 + np.exp(c - a * v ** 2)))
        res = res * partial_res
    return res

In [47]:
x = np.array([1, 1])

interest(x)

0.04800000000000001

In [65]:
from networkx import sigma


def interest(y_hat: np.ndarray) -> float:
    interest_region = {'Pg_f': [0.2, 0.3],
                       'Tg_Tmax': [0.4, 0.7]}

    center = np.array([np.mean(interest_region[ir]) for ir in interest_region.keys()])
    sigma = np.diag([(interest_region[ir][1] - interest_region[ir][0]) / 2 for ir in interest_region.keys()])

    if y_hat.shape == (2,):
        v = np.linalg.solve(sigma, (y_hat - center).reshape(-1, 1))
    else:
        v = np.linalg.solve(sigma, (y_hat - center).reshape(-1, 1)).flatten()

    return 1 - np.exp(-0.5 * np.dot(v.T, v))

In [101]:
def interest(y_hat: np.ndarray) -> float:
    interest_region = {'Pg_f': [0.2, 0.3],
                       'Tg_Tmax': [0.4, 0.7]}

    center = np.array([np.mean(interest_region[ir]) for ir in interest_region.keys()])
    sigma = np.diag([(interest_region[ir][1] - interest_region[ir][0]) / 2 for ir in interest_region.keys()])

    if y_hat.shape == (2,):
        v = np.linalg.solve(sigma, (y_hat - center).reshape(-1, 1))
    else:
        v = np.linalg.solve(sigma, (y_hat - center).reshape(-1, 1)).flatten()

    width_factors = np.array([0.2, 0.8])  # Adjust these factors for each dimension

    # Calculate the squared Mahalanobis distance for each dimension
    mahalanobis_distance_sq = np.dot(v.T * width_factors, v)

    # Bell-shaped curve: Use a quadratic function with dimension-specific width adjustments
    bell_curve = np.exp(-0.5 * mahalanobis_distance_sq)

    # Scale and shift the bell curve to have values between 0 and 1
    scaled_bell_curve = 1 / (1 + bell_curve)

    return scaled_bell_curve


In [102]:
# Make 2d grid of values in range 0 to 1
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
X, Y = np.meshgrid(x, y)

# Calculate interest for each value in grid
Z = np.zeros(X.shape)

for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        Z[i, j] = interest(np.array([X[i, j], Y[i, j]]))

In [103]:
import plotly.graph_objects as go
from cProfile import label
import plotly.io as pio

pio.renderers.default = "browser"


In [104]:
fig = go.Figure(data=[go.Surface(z=Z, x=X, y=Y)], layout=go.Layout(scene=dict(xaxis_title='Pg_f', yaxis_title='Tg_Tmax', zaxis_title='Interest')))
fig.show()

